In [1]:
from glob import glob
import os
# from pathlib import Path
from collections import Counter

from tqdm import tqdm
import pandas as pd
# from wordcloud import WordCloud
# from konlpy.tag import Twitter
# from dotenv import load_dotenv
from openai import OpenAI
# from sqlalchemy import text, create_engine

In [ ]:
pip install pyarrow

In [2]:
# dotenv_path = Path('..') / '.env'
# load_dotenv(dotenv_path=dotenv_path)

# db_host = os.environ["DB_HOST"]
# db_user = os.environ['DB_USER']
# db_password = os.environ['DB_PASSWORD']
# db_name = os.environ['DB_NAME']

# engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:5432/{db_name}')

In [3]:
# query = 'SELECT * FROM review_raw'

# # 데이터 다운로드
# df = pd.read_sql(query, engine)

# # 데이터 확인
# display(df)
# df.info()

In [4]:
df = pd.concat([pd.read_parquet(path) for path in glob("../outputs/*.parquet")])

In [5]:
df = pd.concat([group.head(10) for _, group in df.groupby("store_name")])

In [6]:
def get_gpt_result(msg):


    api_key = ""
    client = OpenAI(api_key=api_key)

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": msg
            }
        ]
    )

    return completion.choices[0].message.content


In [7]:
chef_exists = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    msg = f"""
아래의 리뷰에서 해당 식당의 셰프인 {row.chef_name}가 있었는지 알려줘
방문한 사람이 위 셰프를 봤다면 True, 보지 못했다면 False, 알 수 없다면 NULL을 뽑아줘
다른 번잡한 말 필요없이 딱 Boolean만 줘야해. 반드시!

{row.review_text}
"""
    result = get_gpt_result(msg)
    chef_exists.append(result)


100%|██████████| 30/30 [00:18<00:00,  1.60it/s]


In [8]:
df = df.assign(chef_exists = chef_exists)

In [9]:
df.head()

,user_name,keywords,review_text,review_dt,raw_text,chef_name,store_name,scraped_at,chef_exists
0,동화구연,"[예약 후 이용, 대기 시간 바로 입장, 데이트, 연인・배우자]",7시 예약 후 20분 일찍 도착했는데 바로 입장 가능했습니다.동파육 - 엄청난 기대...,11.7.목,동화구연리뷰 55사진 103팔로우이전다음예약 후 이용대기 시간 바로 입장데이트연인・...,철가방,도량,2024-11-10 09:52:07.275306,True
1,치킨헤드,"[예약 후 이용, 대기 시간 2시간 이상, 여행, 친구]",★ ★ ★ ★ ★ 완벽 그 잡채.맛+친절+분위기 모두 잡음.쉐프님 정말 친절.토요일...,10.26.토,치킨헤드리뷰 118사진 116팔로워 16팔로우예약 후 이용대기 시간 2시간 이상여행...,철가방,도량,2024-11-10 09:52:07.275306,True
2,김밥유령,"[예약 없이 이용, 대기 시간 2시간 이상, 나들이, 연인・배우자]","동파육, 양고기튀김, 차돌짬뽕, 해산물덮밥 주문했어요. 다른 중식집은 자극적이고 ...",10.9.수,"김밥유령리뷰 1,431사진 1,577팔로워 45팔로우예약 없이 이용대기 시간 2시간...",철가방,도량,2024-11-10 09:52:07.275306,False
3,98로라,"[예약 없이 이용, 대기 시간 2시간 이상, 친목, 연인・배우자, 친구]",주말 오픈런 일등하고 왔슴다.새벽4시40분 도착해서 바로 대기했슴다주문한메뉴는 동파...,10.9.수,"98로라리뷰 1,449사진 989팔로워 112테마 9팔로우예약 없이 이용대기 시간 ...",철가방,도량,2024-11-10 09:52:07.275306,False
4,Start21,"[예약 후 이용, 대기 시간 2시간 이상, 데이트, 연인・배우자]",웨이팅 순번이 끝자락이라 동파육은 맛볼 수 없을 줄 알았는데 먹어볼 수 있어서 좋았...,10.27.일,Start21리뷰 39사진 58팔로우예약 후 이용대기 시간 2시간 이상데이트연인・배...,철가방,도량,2024-11-10 09:52:07.275306,True


In [ ]:
df.to_excel("셰프존재결과.xlsx", index=False)